In [2]:

import fastf1
import os

In [1]:


#this function is going to be used to check what platform I'm running this on so I dont have to put the data file on github
cache_path = r'./dataFileDesktop'
while False:
    # Create the cache directory if it doesn't exist
    if not os.path.exists(cache_path):
        os.makedirs(cache_path)

    if os.path.isfile(r'C:\Programming\SimplyLovelyF1\dataFileDesktop'):
        file = r'C:\Programming\SimplyLovelyF1\dataFileDesktop'
    else:
        print("not on your desktop bro")


In [2]:
import os
import fastf1
import pandas as pd
import time

# Set up cache path
cache_path = r'./dataFileDesktop'
if not os.path.exists(cache_path):
    os.makedirs(cache_path)

fastf1.Cache.enable_cache(cache_path)

# Years and sessions to include
years = [2023]
practice_sessions = ['FP1', 'FP2', 'FP3']
f1_2023_races = [
    'Brazil',            # Brazilian Grand Prix
    'Las Vegas',         # Las Vegas Grand Prix
    'Abu Dhabi'          # Abu Dhabi Grand Prix
]

all_laps = []

successful_races = set()
failed_races = set()

for year in years:
    try:
        schedule = fastf1.get_event_schedule(year, include_testing=False)
        for _, event in schedule.iterrows():
            location = event['EventName']
            if location not in f1_2023_races:
                print(f"⚠️ Skipping {location} as it's not in the 2023 race list")
                continue
            race_success = False
            for practice in practice_sessions:
                try:
                    session = fastf1.get_session(year, location, practice)
                    session.load()
                    laps = session.laps
                    if laps.empty:
                        print(f"No laps for {year} {location} {practice}")
                        continue

                    # Merge with weather data if available
                    if session.weather_data is not None and not session.weather_data.empty:
                        laps = laps.merge(session.weather_data, on='Time', how='left')
                    else:
                        laps['TrackTemperature'] = pd.NA
                        laps['Rainfall'] = pd.NA

                    # Add missing columns if not present
                    if 'TrackTemperature' not in laps.columns:
                        laps['TrackTemperature'] = pd.NA
                    if 'Rainfall' not in laps.columns:
                        laps['Rainfall'] = pd.NA

                    # Extract only necessary columns
                    subset = laps[['Driver', 'Team', 'LapNumber', 'LapTime',
                                   'TrackTemperature', 'Rainfall']].copy()
                    subset['Year'] = year
                    subset['Track'] = location

                    all_laps.append(subset)
                    print(f"✅ Added: {year} {location} {practice} - {len(subset)} laps")

                except Exception as e:
                    print(f"⚠️ Skipped {year} {location} {practice}: {e}")
                print("Sleeping for 5 seconds between API calls...")
                time.sleep(5) 
                if race_success:
                    successful_races.add(location)
        else:
            failed_races.add(location)
    except Exception as e:
        print(f"⚠️ Skipped year {year}: {e}")
        print("Sleeping for 5 seconds before next year...")
        time.sleep(5)  # 5 second delay between years

# Concatenate all collected lap data
if all_laps:
    df = pd.concat(all_laps, ignore_index=True)
    df = df[['Year', 'Track', 'Team', 'Driver', 'Rainfall', 'TrackTemperature', 'LapNumber', 'LapTime']]
    
    # Check if file exists and append if it does
    if os.path.exists('practice_laps_2020_2023.csv'):
        # Read existing data
        existing_df = pd.read_csv('practice_laps_2020_2023.csv')
        # Combine with new data
        combined_df = pd.concat([existing_df, df], ignore_index=True)
        # Remove any duplicates
        combined_df = combined_df.drop_duplicates()
        # Save the combined data
        combined_df.to_csv('practice_laps_2020_2023.csv', index=False)
        print("\n✅ Data appended to practice_laps_2020_2023.csv")
    else:
        # If file doesn't exist, create new file
        df.to_csv('practice_laps_2020_2023.csv', index=False)
        print("\n✅ New data saved to practice_laps_2020_2023.csv")
else:
    print("\n❌ No lap data was collected.")

# Print summary of successful and failed races
print("\n=== Data Collection Summary ===")
print(f"Successful races: {len(successful_races)}")
print(f"Failed races: {len(failed_races)}")
if failed_races:
    print("Failed races:", ", ".join(failed_races))

⚠️ Skipping Bahrain Grand Prix as it's not in the 2023 race list
⚠️ Skipping Saudi Arabian Grand Prix as it's not in the 2023 race list
⚠️ Skipping Australian Grand Prix as it's not in the 2023 race list
⚠️ Skipping Azerbaijan Grand Prix as it's not in the 2023 race list
⚠️ Skipping Miami Grand Prix as it's not in the 2023 race list
⚠️ Skipping Monaco Grand Prix as it's not in the 2023 race list
⚠️ Skipping Spanish Grand Prix as it's not in the 2023 race list
⚠️ Skipping Canadian Grand Prix as it's not in the 2023 race list
⚠️ Skipping Austrian Grand Prix as it's not in the 2023 race list
⚠️ Skipping British Grand Prix as it's not in the 2023 race list
⚠️ Skipping Hungarian Grand Prix as it's not in the 2023 race list
⚠️ Skipping Belgian Grand Prix as it's not in the 2023 race list
⚠️ Skipping Dutch Grand Prix as it's not in the 2023 race list
⚠️ Skipping Italian Grand Prix as it's not in the 2023 race list
⚠️ Skipping Singapore Grand Prix as it's not in the 2023 race list
⚠️ Skipping 

In [4]:
session = fastf1.get_session(2023, "Abu Dhabi", "FP1")
session.load()
print(f"{session.event.year} Season Round {session.event.round}: {session.event.name} - {session.name}")


core           INFO 	Loading data for Abu Dhabi Grand Prix - Practice 1 [v3.5.3]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INF

2023 Season Round <bound method Series.round of RoundNumber                                                         22
Country                                                      Abu Dhabi
Location                                                    Yas Island
OfficialEventName    FORMULA 1 ETIHAD AIRWAYS ABU DHABI GRAND PRIX ...
EventDate                                          2023-11-26 00:00:00
EventName                                         Abu Dhabi Grand Prix
EventFormat                                               conventional
Session1                                                    Practice 1
Session1Date                                 2023-11-24 13:30:00+04:00
Session1DateUtc                                    2023-11-24 09:30:00
Session2                                                    Practice 2
Session2Date                                 2023-11-24 17:00:00+04:00
Session2DateUtc                                    2023-11-24 13:00:00
Session3                     

In [4]:
import pandas as pd

file = "f1_lapData_2020-2025.csv"

df_raw = pd.read_csv(file)


In [6]:
df_raw["FP1 Lap Time"]


0                           NaN
1        0 days 00:01:37.766000
2                           NaN
3                           NaN
4        0 days 00:01:34.783000
                  ...          
65079                       NaN
65080                       NaN
65081                       NaN
65082                       NaN
65083                       NaN
Name: FP1 Lap Time, Length: 65084, dtype: object